# Transformer모델을 이용한 요약 만들기

## 1. 데이터 확인

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

import re
import os
import io
import time
import random

import seaborn # Attention 시각화를 위해 필요!
from sklearn.model_selection import train_test_split
import glob

In [3]:
path1 = os.getenv('HOME') + '/aiffel/Deep_Sum/104/8' # 자신에 맞는 숫자 집어넣기

filenames1 = glob.glob(path1 + "/*.csv")


dfs = []
for filename in filenames1:
    dfs.append(pd.read_csv(filename))


data = pd.concat(dfs, ignore_index=True)
print(data.shape)

(111539, 7)


In [4]:
data = data.drop_duplicates(['contents'], ignore_index = True)

In [5]:
text = data['contents']
title = data['title']
print(title.head())
print(text.head())
print(len(text))

0                                   SPAIN WEATHER
1                      유엔기구 "레바논 인구의 55% 이상 빈곤상태"
2    [미 민주 전대] 질 바이든 "해리스 강한여성, 맹렬·터프…남편 강한여성 존경"
3                WTO "내년 세계무역 전망, V자형보다 L자형이 현실적"
4               질 바이든, 트럼프 '바이든 치매' 공격에 "어처구니 없어"
Name: title, dtype: object
0    기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...
1    8월 10일(현지시간) 레바논 수도 베이루트에서 벌어진 반정부 시위[AFP=연합뉴스...
2    미국 민주당 전당대회에서 찬조연설한 질 바이든 여사와 대통령후보 조 바이든 전 부통...
3    6월 상품교역지수\n\n[사진=WTO 제공. 재판매 및 DB 금지] 6월 상품교역지...
4    [ AP/뉴시스] 조 바이든(왼쪽) 전 미국 부통령이 18일(현지시간) 화상으로 진...
Name: contents, dtype: object
32935


In [6]:
cleaning_text = []
for t in text:
    pattern = '[\r|\n|\n\n]'
    a = re.sub(pattern = pattern, repl= ' ', string = t)
    
    cleaning_text.append(a)

In [7]:
cleaning_text[4]

'[ AP/뉴시스] 조 바이든(왼쪽) 전 미국 부통령이 18일(현지시간) 화상으로 진행된 민주당 전당대회에서 대통령후보로 공식선출된 후 부인 질 바이든 여사와 함께 활짝 웃고 있다. 2020.08.19 [ AP/뉴시스] 조 바이든(왼쪽) 전 미국 부통령이 18일(현지시간) 화상으로 진행된 민주당 전당대회에서 대통령후보로 공식선출된 후 부인 질 바이든 여사와 함께 활짝 웃고 있다. 2020.08.19  [서울=뉴시스] 신정원 기자 = 조 바이든 미국 민주당 대선 후보 아내 질 바이든 여사는 19일(현지시간) 남편에 대한 도널드 트럼프 대통령의 \'치매\' 공격에 대해 "어처구니 없다(ridiculous)"고 응수했다.바이든 여사는 이날 NBC \'투데이 쇼\'와의 인터뷰에서 트럼프 대통령이 바이든 후보에 대해 \'인지 능력 검사를 받아야 한다\'고 공격하는 것이 정당하냐는 진행자의 질문에 "아니다. 어처구니 없다"며 "조는 그와 통화하길 원하는 주지사들이나 낸시 펠로시 하원의장과 매분 매초 전화통화를 하고 있다"고 답했다.이어 "그는 주목받고 있으며 기금 모금도 하고 브리핑도 한다"면서 "오전 9시부터 오후 11시까지 멈추지 않는다. 알다시피, (트럼프 대통령의 주장은) 말도 안 된다"고 강조했다.트럼프 대통령 재선 캠프는 77세인 바이든 후보가 인지 능력 검사를 받아야 한다고 공격해 왔다. 18일엔 이를 소재로 한 \'바이든에게 무슨 일이 있었나\' 제목의 광고를 내보내기도 했다.더힐에 따르면 지난달 몬머스대 여론조사에선 응답자의 52%가 바이든 후보가 정신적, 육체적으로 미 대통령직을 수행할 수 있다고 믿는다고 답했다. 트럼프 대통령에 대해선 45%만 같은 의견을 냈다. 트럼프 대통령은 바이든 후보보다 3살 아래인 74세다.바이든 후보는 18일 민주당 화상 전당대회에서 민주당 대통령 후보로 공식 선출됐다. 그는 오는 11월 대선에서 승리하면 최고령 미 대통령 당선자가 된다. ☞공감언론 뉴시스 jwshin@newsis.com'

In [8]:
with open('text.txt', 'w') as f:
    for text in cleaning_text:
        f.writelines('{}\n'.format(text))
title.to_csv('title.txt', index = False, header = None, sep ='\t')

In [9]:
title_path = os.getenv('HOME') + '/aiffel/Deep_Sum/title_104.txt'
text_path = os.getenv('HOME') + '/aiffel/Deep_Sum/text_104.txt'

In [10]:
with open(title_path, 'r' ) as f:
    title= f.read().splitlines()
    
print(title[4])

with open(text_path, 'r') as f:
    text = f.read().splitlines()


print(text[4])

"질 바이든, 트럼프 '바이든 치매' 공격에 ""어처구니 없어"""
[ AP/뉴시스] 조 바이든(왼쪽) 전 미국 부통령이 18일(현지시간) 화상으로 진행된 민주당 전당대회에서 대통령후보로 공식선출된 후 부인 질 바이든 여사와 함께 활짝 웃고 있다. 2020.08.19 [ AP/뉴시스] 조 바이든(왼쪽) 전 미국 부통령이 18일(현지시간) 화상으로 진행된 민주당 전당대회에서 대통령후보로 공식선출된 후 부인 질 바이든 여사와 함께 활짝 웃고 있다. 2020.08.19  [서울=뉴시스] 신정원 기자 = 조 바이든 미국 민주당 대선 후보 아내 질 바이든 여사는 19일(현지시간) 남편에 대한 도널드 트럼프 대통령의 '치매' 공격에 대해 "어처구니 없다(ridiculous)"고 응수했다.바이든 여사는 이날 NBC '투데이 쇼'와의 인터뷰에서 트럼프 대통령이 바이든 후보에 대해 '인지 능력 검사를 받아야 한다'고 공격하는 것이 정당하냐는 진행자의 질문에 "아니다. 어처구니 없다"며 "조는 그와 통화하길 원하는 주지사들이나 낸시 펠로시 하원의장과 매분 매초 전화통화를 하고 있다"고 답했다.이어 "그는 주목받고 있으며 기금 모금도 하고 브리핑도 한다"면서 "오전 9시부터 오후 11시까지 멈추지 않는다. 알다시피, (트럼프 대통령의 주장은) 말도 안 된다"고 강조했다.트럼프 대통령 재선 캠프는 77세인 바이든 후보가 인지 능력 검사를 받아야 한다고 공격해 왔다. 18일엔 이를 소재로 한 '바이든에게 무슨 일이 있었나' 제목의 광고를 내보내기도 했다.더힐에 따르면 지난달 몬머스대 여론조사에선 응답자의 52%가 바이든 후보가 정신적, 육체적으로 미 대통령직을 수행할 수 있다고 믿는다고 답했다. 트럼프 대통령에 대해선 45%만 같은 의견을 냈다. 트럼프 대통령은 바이든 후보보다 3살 아래인 74세다.바이든 후보는 18일 민주당 화상 전당대회에서 민주당 대통령 후보로 공식 선출됐다. 그는 오는 11월 대선에서 승리하면 최고령 미 대통령 당선자가 된다. ☞공감언론 뉴시스 jwshin@n

In [11]:
print(len(text))
print(len(title))
print(type(text[0]))
print(type(title[0]))

102248
102248
<class 'str'>
<class 'str'>
